In [70]:
import torch
import pickle
import random

from sklearn.decomposition import PCA



class close_words:
    def __init__(self,model,w2i,pca=False,n=5):
        self.model=model
        self.w2i=w2i
        self.n=n
        self.i2w=dict(zip(w2i.values(),w2i.keys()))
        pca = PCA(n_components=5)
        self.embeddings=model['embedding_in.weight'] if not pca else torch.tensor(pca.fit_transform(model['embedding_in.weight']))
        
        
    def input_word(self,word):
        self.sample_idx=self.w2i[word]
        self.sample_vec=self.embeddings[self.sample_idx]
    
    def sample_word(self):
        
        self.sample_idx=random.sample(list(self.w2i.values()),1)[0]
        self.sampled_word=self.i2w[self.sample_idx]
        self.sample_vec=self.embeddings[self.sample_idx]
        print('임의의 단어:',self.sampled_word)
    
    def l2_dist(self):
        
        self.trial=torch.mul((self.embeddings-self.sample_vec),(self.embeddings-self.sample_vec))
        self.trial[self.sample_idx]=100
        self.trial=torch.sum(self.trial,dim=1)
        #rec=torch.argmax(-self.trial).item()
        idexes=torch.argsort(-self.trial, descending=True)[:self.n]
        step=1
        print(len(self.i2w))
        for idx in idexes:
            if idx==len(w2i): #unk 제거
                continue
            print(step,'번째 가까운 단어:',self.i2w[idx.item()])
            step+=1
                
    def cos_sim(self):
      
        self.trial=torch.div(torch.sum(torch.mul(self.embeddings,self.sample_vec),dim=1),torch.sum(torch.mul(self.embeddings,self.embeddings),dim=1))
     
        self.trial[self.sample_idx]=-100
        idexes=torch.argsort(self.trial, descending=True)[:self.n]
        step=1
        for idx in idexes:
            if idx==len(w2i)+1: #unk 제거
                continue
            print(step,'번째 가까운 단어:',self.i2w[idx.item()])
            step+=1
        

In [71]:
#model=torch.load('./w2v_withoud_ns',map_location='cpu')
model=torch.load('./training_with_mecab_2_epoch_7',map_location='cpu')['state_dict']

with open('./preprocessing//vocab_mecab.pickle','rb') as f:
    w2i=pickle.load(f)
    
print('임베딩 사이즈:',model['embedding_in.weight'].shape)  #unk 랑 embedding이 0부터인것을 몰랐음. 0은 버리면 됨
print('사전 사이즈:',len(w2i))  

임베딩 사이즈: torch.Size([22795, 256])
사전 사이즈: 22794


In [77]:
showing=close_words(model,w2i,pca=True,n=10)
showing.sample_word()
showing.l2_dist()
print('임의의 단어',showing.sampled_word)
showing.cos_sim()

임의의 단어: ('보관', 'NNG')
22794
1 번째 가까운 단어: ('일원', 'NNG')
2 번째 가까운 단어: ('지양', 'NNG')
3 번째 가까운 단어: ('주지', 'NNG')
4 번째 가까운 단어: ('시츠', 'NNP')
5 번째 가까운 단어: ('냄새', 'NNG')
6 번째 가까운 단어: ('시골', 'NNG')
7 번째 가까운 단어: ('웠어요강추합니당', 'UNKNOWN')
8 번째 가까운 단어: ('뽑히', 'VV')
9 번째 가까운 단어: ('승차', 'NNG')
10 번째 가까운 단어: ('맞물리', 'VV')
임의의 단어 ('보관', 'NNG')
1 번째 가까운 단어: ('배연재', 'NNP')
2 번째 가까운 단어: ('유성', 'NNG')
3 번째 가까운 단어: ('아실', 'VV+EP+ETM')
4 번째 가까운 단어: ('프리미어', 'NNP')
5 번째 가까운 단어: ('notation', 'SL')
6 번째 가까운 단어: ('모범', 'NNG')
7 번째 가까운 단어: ('Hawaiian', 'SL')
8 번째 가까운 단어: ('서면', 'NNG')
9 번째 가까운 단어: ('대할', 'VV+ETM')
10 번째 가까운 단어: ('제자', 'NNG')


In [98]:
showing.input_word(('중간', 'NNG'))
showing.l2_dist()
showing.cos_sim()

22794
1 번째 가까운 단어: ('학점', 'NNG')
2 번째 가까운 단어: ('기말고사', 'NNG')
3 번째 가까운 단어: ('입니다', 'VCP+EF')
4 번째 가까운 단어: ('받', 'VV')
5 번째 가까운 단어: ('같', 'VA')
6 번째 가까운 단어: ('번', 'NNBC')
7 번째 가까운 단어: ('않', 'VX')
8 번째 가까운 단어: ('내용', 'NNG')
9 번째 가까운 단어: ('있', 'VX')
10 번째 가까운 단어: ('때문', 'NNB')
1 번째 가까운 단어: ('화남', 'NNG')
2 번째 가까운 단어: ('이상욱', 'NNP')
3 번째 가까운 단어: ('써온', 'VV+EC+VX+ETM')
4 번째 가까운 단어: ('정순', 'NNG')
5 번째 가까운 단어: ('정교수', 'NNG')
6 번째 가까운 단어: ('공급', 'NNG')
7 번째 가까운 단어: ('못함', 'VX+ETN')
8 번째 가까운 단어: ('advance', 'SL')
9 번째 가까운 단어: ('20000', 'SN')
10 번째 가까운 단어: ('신뢰', 'NNG')


## UNK 빈도수

In [98]:
with open('./preprocessing/vocab_without_josa_gut_su_freq.pickle','rb') as f:
    total_freq=pickle.load(f)
    
with open('./preprocessing/vocab_without_josa_gut_su.pickle','rb') as h:
    voc=pickle.load(h)
    
not_unk=list(voc.keys())

freq_sum=0

def summing(x):
    
    global freq_sum
    freq_sum+=x

_=list(map(lambda x:summing(total_freq[x]),not_unk))

ratio=(sum(total_freq.values())-freq_sum)/sum(total_freq.values())
print(ratio)

0.014246669941204219


In [102]:
## unk 빈도수가 문제는 아닌듯??